In [1]:
from tensorflow import keras

In [2]:
from keras import Sequential
from keras.layers import Dense, Flatten, Conv1D, Dropout, Embedding, MaxPooling1D, Conv2D

Using TensorFlow backend.


In [3]:
import joblib
import numpy as np

##### DATA

In [4]:
with open(r'F:\ОБУЧЕНИЕ\CHAT-BOTs\тексты\dialogues\_phrases2.txt', encoding='utf-8') as f:
    phrases = f.readlines()
phrases = [i.replace('\n', '') for i in phrases]
with open(r'F:\ОБУЧЕНИЕ\CHAT-BOTs\тексты\dialogues\_intents2.txt', encoding='utf-8') as f:
    intents = f.readlines()    
intents = [i.replace('\n', '') for i in intents]

In [5]:
from gensim.models import Word2Vec
from keras.preprocessing.text import text_to_word_sequence

def preproc(phrase):
    return text_to_word_sequence(phrase, 
                              filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                              lower=True,
                              split=' ')

text = [preproc(phrase) for phrase in phrases]
wvec = Word2Vec(text, size=100, window=5, min_count=3, workers=4, )

In [6]:
def vectorize(phrase):
    temp_vector=[]
    if isinstance(phrase, list)==False:
        phrase=preproc(phrase)
    for word in phrase:
        if word not in wvec.wv:
            temp_vector.append([0.0]*100)
        else:
            temp_vector.append(wvec.wv.__getitem__(word))
    result = np.mean(temp_vector, axis=0)
    if result.all():
        return result
    else:
        return np.array([0.0]*100)
#     return [0.0]*100 if np.mean(temp_vector, axis=0)==None else 1

X = [vectorize(phrase) for phrase in text]
len(X)

539

In [7]:
X[0]

array([-5.0708215e-04,  3.0788551e-03,  3.2656330e-03,  3.1042786e-03,
        2.3811192e-03,  2.1826783e-03,  3.3844253e-03, -2.7886888e-03,
        1.3388204e-03,  1.7219025e-03,  2.4272897e-03, -2.7946853e-03,
       -1.8071223e-03,  4.5693121e-03, -2.7696525e-03, -4.3898788e-03,
       -6.8755355e-04,  2.4962876e-04, -1.8648905e-03, -3.6752785e-03,
        3.2827733e-03,  3.4923719e-03, -8.1032835e-04, -2.6132232e-03,
        2.6049740e-03,  1.1201703e-03, -4.7834525e-03, -3.5013445e-03,
       -1.5503314e-03, -2.0130472e-03, -9.9260896e-04, -5.6130585e-04,
        2.6451771e-03, -7.9949369e-04, -2.9676724e-03, -1.3836005e-04,
       -3.6652186e-03,  6.2864181e-04,  2.6051805e-05, -2.1191672e-03,
       -1.7992903e-03, -2.1717281e-03,  1.3824430e-03, -3.7001714e-03,
        2.9474176e-03,  1.2255094e-03, -1.1858565e-03, -1.9779895e-03,
        4.7480329e-03,  4.4397363e-03,  4.7666575e-03,  2.5824313e-03,
       -4.2345305e-03, -2.9056596e-03,  6.1519834e-04, -2.3504216e-03,
      

In [8]:
X[0].shape

(100,)

In [9]:
classes = list(set(intents))
y = [classes.index(i) for i in intents]

In [10]:
y[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [11]:
len(y)

539

##### split to train/test samples

In [12]:
from keras.utils import to_categorical
y_binary = to_categorical(y)

In [13]:
y_binary

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.array(X), y_binary, test_size=0.2, random_state=444)

In [15]:
len(x_train)

431

In [16]:
x_train[:2]

array([[ 1.58241251e-03,  1.03852875e-03,  1.21985900e-03,
        -2.35863565e-03,  3.36328335e-03,  1.07771065e-03,
        -3.06543894e-03, -9.54213960e-04,  1.05745066e-03,
        -1.29183498e-03,  2.25803815e-05, -2.31243717e-03,
         2.59578461e-03, -3.84419342e-03, -1.76319061e-03,
         2.29572295e-03,  1.96145033e-04, -1.66256947e-03,
         9.74284834e-04, -6.94119954e-05,  1.11607299e-03,
        -1.53306697e-03,  6.94769551e-04,  1.16062514e-03,
         2.69782241e-03,  4.91414499e-03,  1.40768406e-03,
        -3.94850411e-03,  1.30934897e-03,  9.68409353e-04,
         7.94305815e-04,  1.56655046e-03, -2.22493079e-04,
         1.11214898e-03,  2.98105879e-03, -6.24118722e-04,
        -2.19683000e-03,  1.28916244e-03,  2.10014754e-03,
        -3.79301002e-03,  8.09239573e-04,  1.21054868e-03,
        -3.72492883e-04,  3.65307718e-03,  3.18812998e-03,
        -4.00296785e-03,  2.39610393e-03,  4.17943695e-04,
        -3.95043753e-03,  1.21470238e-03,  3.50838481e-0

##### positions of words

In [48]:
# объединим в один список все предложения
full_list = []
for phrase in text:
    full_list += phrase
# составим список уникальных слов
words_list = list(set(full_list))
# создадим словарь слово в номер
word2index = {v:k for k,v in enumerate(words_list)}
word2index.update({'unknown':-1})
# создадим обратный словарь
index2word = {v:k for k,v in word2index.items()}
# создадим список текстов, где вместо слова его индекс
def text_to_word_indexes(phrase):
    temp_list=[]
    for word in phrase:
        if word in word2index:
            temp_list.append(word2index[word])
        else:
            word='unknown'
            temp_list.append(word2index[word])
    return temp_list

text_indexes = []
for phrase in text:
    text_indexes.append(np.asarray(text_to_word_indexes(phrase)))

In [49]:
text_indexes[:5]

[array([61]), array([42, 65]), array([63]), array([81]), array([55])]

In [19]:
from keras.preprocessing.sequence import pad_sequences

In [20]:
max_len_sequnce=0
for phrase in text_indexes:
    if len(phrase)>max_len_sequnce:
        max_len_sequnce=len(phrase)
max_len_sequnce

7

In [21]:
pad_sequences([text_indexes[1]], maxlen=max_len_sequnce, padding='post')

array([[42, 65,  0,  0,  0,  0,  0]])

In [22]:
text

[['привет'],
 ['добрый', 'день'],
 ['приветик'],
 ['здарова'],
 ['хай'],
 ['hi'],
 ['hello'],
 ['добрый', 'вечер'],
 ['доброй', 'ночи'],
 ['доброе', 'утро'],
 ['пока'],
 ['счастливо', 'оставаться'],
 ['счастливо'],
 ['адьес'],
 ['bye'],
 ['bye', 'bye'],
 ['до', 'новых', 'встреч'],
 ['до', 'встречи'],
 ['как', 'дела'],
 ['как', 'твои', 'дела'],
 ['как', 'делишки'],
 ['как', 'оно'],
 ['как', 'жизнь'],
 ['че', 'как'],
 ['как', 'сам'],
 ['расскажи', 'анекдот'],
 ['пошути', 'как', 'нибудь'],
 ['знаешь', 'приколы'],
 ['напиши', 'что', 'нибудь', 'смешное'],
 ['анекдот', 'хочу'],
 ['шутки', 'знаешь'],
 ['шутить', 'умеешь'],
 ['знаешь', 'какой', 'нибудь', 'прикол'],
 ['сделай', 'рассылку', 'студентам'],
 ['пошли', 'студентам', 'такой', 'текст'],
 ['отправь', 'информацию', 'студентам'],
 ['разошли', 'письма', 'с', 'новостью'],
 ['какая', 'погода'],
 ['что', 'за', 'окном'],
 ['будет', 'ли', 'сегодня', 'дождь'],
 ['сегодня', 'солнечно'],
 ['сколько', 'градусов'],
 ['температуру', 'можешь', 'сказат

In [23]:
text_to_word_indexes(['привет'])

[61]

In [50]:
def pad_seq(phrase):
    return pad_sequences([text_to_word_indexes(phrase)], maxlen=max_len_sequnce+1, padding='post')[0]

X = []
for phrase in text:
    X.append(pad_seq(phrase))
X=np.asarray(X)
X[:2]

array([[61,  0,  0,  0,  0,  0,  0,  0],
       [42, 65,  0,  0,  0,  0,  0,  0]])

In [25]:
X.shape

(539, 7)

##### labels

In [26]:
classes = list(set(intents))
y = [classes.index(i) for i in intents]

In [27]:
y[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [28]:
len(y)

539

##### split to train/test samples

In [29]:
from keras.utils import to_categorical
y_binary = to_categorical(y)

In [30]:
y_binary

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [51]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=444)

##### MODEL

In [52]:
input_shape = (None,300)

In [60]:
model = Sequential()
model.add(Embedding(100, 64, input_length=max_len_sequnce+1))
# Embedding(len(words_list), 7, input_length=max_len_sequnce)
# model.add(Conv2D(64, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=(input_shape)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D())
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(7, activation='softmax', kernel_regularizer=keras.regularizers.l1(1e-8)))
# adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [61]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 8, 64)             6400      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 6, 64)             12352     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 3, 64)             0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 192)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)                6176      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 16)               

In [62]:
model.fit(x_train
          ,np.array(y_train)
          ,batch_size=10
          ,epochs=40
          ,verbose=1 
#           ,validation_split=0.2 
          ,validation_data=(x_test, np.array(y_test))
          ,shuffle=True
         )

C:\Users\vndan\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 431 samples, validate on 108 samples
Epoch 1/40
431/431 [==============================] - 1s 1ms/step - loss: 1.9078 - accuracy: 0.2158 - val_loss: 1.8138 - val_accuracy: 0.3611
Epoch 2/40
431/431 [==============================] - 0s 757us/step - loss: 1.6600 - accuracy: 0.4130 - val_loss: 1.3490 - val_accuracy: 0.5000
Epoch 3/40
431/431 [==============================] - 0s 758us/step - loss: 1.1812 - accuracy: 0.5197 - val_loss: 0.8747 - val_accuracy: 0.5833
Epoch 4/40
431/431 [==============================] - 0s 753us/step - loss: 0.8153 - accuracy: 0.6705 - val_loss: 0.4827 - val_accuracy: 0.8611
Epoch 5/40
431/431 [==============================] - 0s 764us/step - loss: 0.5943 - accuracy: 0.7633 - val_loss: 0.3084 - val_accuracy: 0.9630
Epoch 6/40
431/431 [==============================] - 0s 745us/step - loss: 0.4130 - accuracy: 0.8747 - val_loss: 0.1824 - val_accuracy: 1.0000
Epoch 7/40
431/431 [==============================] - 0s 750us/step - loss: 0.3697 - accurac

In [63]:
model.evaluate(np.array(x_train),
              y_train,
              batch_size=10, 
              verbose=1)

431/431 [==============================] - 0s 403us/step


[0.00016091085967937055, 1.0]

In [70]:
classes[np.argmax(model.predict(np.array(pad_seq('какая погода брат браза бро йоу какая погода'.split())).reshape(1,-1)))]

'узнать метеопрогноз на сегодня'

In [73]:
np.array(pad_seq('какая погода брат браза бро йоу какая погода'.split())).reshape(1,-1)

array([[71, 80, -1, -1, -1, -1, 71, 80]])